In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import os
os.chdir('../')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import tcav
from src.download_data import download_imagenet, download_random_imagenet_classes
import pickle
from src.dataset import *
from src.concept_vectors import *
from src.util import *
import random
from src.plots import *
from sklearn.manifold import TSNE
from scipy.spatial.distance import cosine 
import pandas as pd
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import make_blobs
from src.hierarchy import *

2023-01-06 00:17:58.983185: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
load_cem_vectors("xor",0,42).shape

(48, 16)

In [5]:
load_tcav_vectors("soccer",['mixed4c'],experiment_name="unfiled")[0].shape

(2, 100352)

In [6]:
load_label_vectors_simple("has_bill_shape::dagger","cub",seed=42).shape

(1, 4796)

In [7]:
load_cem_vectors_simple('0','xor',seed=42).shape

(48, 16)

In [27]:
h1 = create_hierarchy(create_ward_hierarchy,load_cem_vectors_simple, 'xor',['0','1'],42)

In [28]:
h2 = create_hierarchy(create_linkage_hierarchy,load_tcav_vectors_simple, 'unfiled',['soccer','alga'],42)

In [29]:
h3 = create_hierarchy(create_linkage_hierarchy,load_label_vectors_simple, 'cub',["has_bill_shape::dagger",'has_bill_shape::hooked_seabird'],42)

In [33]:
len(str(h1))

32

In [34]:
len(str(h2))

40

In [35]:
len(str(h3))

82

In [40]:
assert type(h1) == type(h2) == type(h3)